## Classifying MNIST data types

### Import packages

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

c:\Users\Ghost\anaconda3\envs\TF2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import Data

In [2]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

In [3]:
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)

### Scaling MNIST Dataset

In [4]:
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255.
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)
test_data=mnist_test.map(scale)

### Shuffling

In [5]:
BUFFER_SIZE=10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data=shuffled_train_and_validation_data.take(num_validation_samples)

train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

### Batching

In [6]:
BATCH_SIZE=100

train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

In [7]:
validation_inputs,validation_targets=next(iter(validation_data))

## Model

### Outline the Model

In [8]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])

### Choose optimizer and loss function

In [9]:
#custom_optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

### Training

In [10]:
NUM_EPOCHS = 7

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/7
540/540 - 4s - loss: 0.3376 - accuracy: 0.9038 - val_loss: 0.1735 - val_accuracy: 0.9487 - 4s/epoch - 8ms/step
Epoch 2/7
540/540 - 2s - loss: 0.1386 - accuracy: 0.9595 - val_loss: 0.1273 - val_accuracy: 0.9642 - 2s/epoch - 5ms/step
Epoch 3/7
540/540 - 2s - loss: 0.0971 - accuracy: 0.9708 - val_loss: 0.1018 - val_accuracy: 0.9698 - 2s/epoch - 4ms/step
Epoch 4/7
540/540 - 2s - loss: 0.0756 - accuracy: 0.9772 - val_loss: 0.0922 - val_accuracy: 0.9718 - 2s/epoch - 4ms/step
Epoch 5/7
540/540 - 2s - loss: 0.0608 - accuracy: 0.9819 - val_loss: 0.0704 - val_accuracy: 0.9780 - 2s/epoch - 4ms/step
Epoch 6/7
540/540 - 3s - loss: 0.0511 - accuracy: 0.9844 - val_loss: 0.0616 - val_accuracy: 0.9792 - 3s/epoch - 5ms/step
Epoch 7/7
540/540 - 4s - loss: 0.0426 - accuracy: 0.9867 - val_loss: 0.0571 - val_accuracy: 0.9822 - 4s/epoch - 7ms/step


### Test the model

In [11]:
test_loss,test_accuracy=model.evaluate(test_data)

1/1 [==============================] - 0s 311ms/step - loss: 0.0835 - accuracy: 0.9741


In [12]:
print(f'Test Loss : {round(test_loss,2)} Test Accuracy : {round(test_accuracy*100,2)}')

Test Loss : 0.08 Test Accuracy : 97.41
